In [1]:
import os
import sys
from pathlib import Path

# Ensure project root is on sys.path so 'backtester' imports work
cwd = Path(os.getcwd())
candidates = [cwd, cwd.parent, Path("..").resolve()]
for p in candidates:
    if (p / "backtester").exists() and str(p) not in sys.path:
        sys.path.insert(0, str(p))
        break

print("PYTHONPATH set. Using root:", sys.path[0])

PYTHONPATH set. Using root: c:\Users\User\Desktop\Crypto strategy backtest


## Load Data

In [2]:
import pandas as pd

def load_crypto_parquet_data(coin_name: str, timeframe: str = "5m", nM: int = 54, section: str = "UTC") -> pd.DataFrame:
    df = pd.read_parquet(fr'C:\Users\User\Desktop\Crypto\{coin_name}_{timeframe}_{nM}M_{section}.parquet')
    return df
def generate_us_session_bars_info(df, include_holidays: bool = False):
    # 確保時間有時區資訊
    df['dt_ny'] = pd.to_datetime(df['dt_utc'], utc=True).dt.tz_convert('America/New_York')

    # 取日期（當地日曆）
    df['date'] = df['dt_ny'].dt.date
    df['weekday'] = df['dt_ny'].dt.day_name() 
    # 對每天依時間排序並編號
    df = df.sort_values(['date', 'dt_ny']).reset_index(drop=True)
    df['bar_index'] = df.groupby('date').cumcount() + 1  # 第幾根K線，從1開始
    if not include_holidays:
        weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday']
    else:
        weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    df = df.loc[df['weekday'].isin(weekday)]
    # 查看結果
    df.set_index('dt_utc', inplace=True)
    # print(df[['dt_ny', 'date', 'weekday', 'bar_index']].head(5))

    return df
def generate_allday_bars_info(df, include_holidays: bool = True):
# 確保時間有時區資訊
    df['dt_ny'] = pd.to_datetime(df['dt_utc'], utc=True).dt.tz_convert('America/New_York')

    # 取日期（當地日曆）
    df['date'] = df['dt_ny'].dt.date
    df['time'] = df['dt_ny'].dt.time
    df['weekday'] = df['dt_ny'].dt.day_name() 
    # 對每天依時間排序並編號
    df = df.sort_values(['date', 'dt_ny']).reset_index(drop=True)
    df['bar_index'] = df.groupby('date').cumcount() + 1  # 第幾根K線，從1開始
    if not include_holidays:
        weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday']
    else:
        weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    df = df.loc[df['weekday'].isin(weekday)]
    # 查看結果
    df.set_index('dt_utc', inplace=True)
    # print(df[['dt_ny', 'date', 'weekday', 'bar_index']].head(5))

    return df

In [3]:
coin_name = 'BTC'  # 可更改為 'ETH', 'SOL', 'BTC', 'ADA', 'PAXG' 等等
nM = 54
timeframe = "5m"
section = "UTC"

df = load_crypto_parquet_data(coin_name=coin_name, timeframe=timeframe, nM=nM, section=section)
df = generate_allday_bars_info(df, include_holidays=False)


## Split Data

In [19]:
# 分成n等分windows
def split_into_windows(df: pd.DataFrame, n_windows: int, overlap_ratio: float = 0.0) -> list[pd.DataFrame]:
    unique_dates = df.index.unique()
    n_dates = len(unique_dates)
    window_size = n_dates // n_windows
    windows = []
    for i in range(n_windows):
        start_idx = i * window_size
        if i == n_windows - 1:  # 最後一個視窗包含剩餘的日期
            end_idx = n_dates
        if overlap_ratio > 0 and i > 0:
            overlap_size = int(window_size * overlap_ratio)
            start_idx -= overlap_size
        end_idx = start_idx + window_size
        window_dates = unique_dates[start_idx:end_idx]
        window_df = df[df.index.isin(window_dates)].copy()
        windows.append(window_df)
    return windows
def split_into_train_test(df: pd.DataFrame, train_pct: float = 0.7) -> tuple[pd.DataFrame, pd.DataFrame]:
    unique_dates = df.index.unique()
    n_dates = len(unique_dates)
    train_size = int(n_dates * train_pct)
    train_dates = unique_dates[:train_size]
    test_dates = unique_dates[train_size:]
    train_df = df[df.index.isin(train_dates)].copy()
    test_df = df[df.index.isin(test_dates)].copy()
    return train_df, test_df

In [20]:
splited_dfs = split_into_windows(df, n_windows=5, overlap_ratio=0.5)
splited_train_test_dfs = [split_into_train_test(window_df, train_pct=0.7) for window_df in splited_dfs]

## Build Strategy

In [ ]:
from backtester.models import Position, Side, ActionType, BacktestConfig
from backtester.engine import BacktestEngine
from backtester.strategy_base import StrategyContext
from backtester.strategies.ALBO_strategy import ALBOStrategy, ALBOParams
cfg = BacktestConfig(initial_cash=10000, fee_rate=0.0, slippage_bps=0.0, conservative_intrabar=True)
engine = BacktestEngine(cfg)

In [50]:
from typing import Any, Dict, List
from itertools import product
def build_param_combinations(grid: Dict[str, List[Any]]) -> List[Dict[str, Any]]:
    """
    將 param_grid 展開成所有組合，每個組合是一個 dict。
    grid 的值必須是 list/tuple 等 iterable。
    """
    keys = list(grid.keys())
    values_lists = [grid[k] for k in keys]
    combos = [dict(zip(keys, vals)) for vals in product(*values_lists)]
    return combos
def tune_strategy_params(train_df: pd.DataFrame, param_grid: dict, best_n_params: int, engine) -> dict:
    best_params = None
    best_performance = -float('inf')
    params_combinations = build_param_combinations(param_grid)
        
    performance_list = []

    for params in params_combinations:
        strat_params = ALBOParams(**params)
        strat = ALBOStrategy(strat_params)
        train_result = engine.run(train_df, strat)
        performance = train_result.equity_curve.values[-1] - train_result.equity_curve.values[0]  # 總收益作為績效指標
        performance_per_day = performance / (len(train_df.index.unique()))

        if performance_per_day > best_performance:
            best_performance = performance_per_day
        performance_list.append((params, performance_per_day))
    performance_list.sort(key=lambda x: x[1], reverse=True)
    best_params_list = performance_list[:best_n_params]

    return best_params_list
def test_strategy_with_params(test_df: pd.DataFrame, params: dict, engine) -> Any:
    strat_params = ALBOParams(**params)
    strat = ALBOStrategy(strat_params)
    test_result = engine.run(test_df, strat)
    performance = test_result.equity_curve.values[-1] - test_result.equity_curve.values[0]
    performance_per_day = performance / (len(test_df.index.unique()))
    return performance_per_day, test_result

In [51]:

param_grid = {
    'rr': [1, 1.2, 1.5],
    'break_out_series_n': [2, 3, 4, 5]
}
best_n_params = 2
best_params_list = []
for i in range(len(splited_train_test_dfs)):
    print(f"=== Window {i+1} ===")
    best_params = tune_strategy_params(
        train_df=splited_train_test_dfs[i][0],
        param_grid=param_grid,
        best_n_params=best_n_params,
        engine=engine
    )
    print("Best Params:")
    for params, perf in best_params:
        print(f"Params: {params}, Performance per day: {perf}")
    best_params_list.append(best_params)



=== Window 1 ===
Best Params:
Params: {'rr': 1.5, 'break_out_series_n': 5}, Performance per day: 0.008712595290139798
Params: {'rr': 1.2, 'break_out_series_n': 5}, Performance per day: 0.006167444851210203
=== Window 2 ===
Best Params:
Params: {'rr': 1, 'break_out_series_n': 5}, Performance per day: -0.003307267425166236
Params: {'rr': 1.5, 'break_out_series_n': 5}, Performance per day: -0.0033723172720674507
=== Window 3 ===
Best Params:
Params: {'rr': 1.5, 'break_out_series_n': 2}, Performance per day: 0.07676657763505756
Params: {'rr': 1.5, 'break_out_series_n': 3}, Performance per day: 0.07312373931325142
=== Window 4 ===
Best Params:
Params: {'rr': 1.5, 'break_out_series_n': 2}, Performance per day: 0.038743379298571086
Params: {'rr': 1, 'break_out_series_n': 2}, Performance per day: 0.035669519406015565
=== Window 5 ===
Best Params:
Params: {'rr': 1, 'break_out_series_n': 2}, Performance per day: 0.07281965667279865
Params: {'rr': 1.2, 'break_out_series_n': 2}, Performance per da

In [56]:
best_params = [
    ({'rr': 1.5, 'break_out_series_n': 5},{'rr': 1.2, 'break_out_series_n': 5}),
    {'rr': 1, 'break_out_series_n': 5},
    {'rr': 1.5, 'break_out_series_n': 2},
    {'rr': 1.5, 'break_out_series_n': 2},
    {'rr': 1, 'break_out_series_n': 2}
]

for i in range(len(splited_train_test_dfs)):
    print(f"=== Window {i+1} Test Results ===")
    for params, train_perf in best_params_list[i]:
        test_perf, test_result = test_strategy_with_params(
            test_df=splited_train_test_dfs[i][1],
            params=params,
            engine=engine
        ) 
        print(f"Test Result with Params: {params}, performance per day: {test_perf}, performance diff: {test_perf - train_perf}")

=== Window 1 Test Results ===
Test Result with Params: {'rr': 1.5, 'break_out_series_n': 5}, performance per day: 0.0, performance diff: -0.008712595290139798
Test Result with Params: {'rr': 1.2, 'break_out_series_n': 5}, performance per day: 0.0, performance diff: -0.006167444851210203
=== Window 2 Test Results ===
Test Result with Params: {'rr': 1, 'break_out_series_n': 5}, performance per day: 0.0002832059463172882, performance diff: 0.0035904733714835243
Test Result with Params: {'rr': 1.5, 'break_out_series_n': 5}, performance per day: 0.003478300406198724, performance diff: 0.006850617678266175
=== Window 3 Test Results ===
Test Result with Params: {'rr': 1.5, 'break_out_series_n': 2}, performance per day: 0.026826792592094646, performance diff: -0.04993978504296292
Test Result with Params: {'rr': 1.5, 'break_out_series_n': 3}, performance per day: -0.022904824264520313, performance diff: -0.09602856357777173
=== Window 4 Test Results ===
Test Result with Params: {'rr': 1.5, 'bre

In [ ]:
params = ALBOParams(
    break_out_series_n = 3, # BO定義:連續突破K線數
    break_out_n_bars = 10, # 突破區間長度（K線數）
    max_notional_pct = 1.0, # 最大虧損佔資金比例%
    min_qty = 0.001, # 最小BTC下單量
    sl_atr_like = 0.0,  # MVP不做ATR，示範保留欄位
    rr = 2.0,           # TP = SL距離 * rr
    time_exit_bars = 50
)
strategy = ALBOStrategy(params)

result = engine.run(df, strategy)

## 